In [40]:
from __future__ import division, print_function, unicode_literals
import numpy as np
import os

PROJECT_ROOT_DIR='/home/musaup/Documents/Tools/sherlock/OnlyNN'
PROJEC_SAVE_DIR='/home/musaup/Documents/tests'
FILE_NAME='neural_network_information_15'
#FILE_NAME='neural_network_information'

Create an open file function

In [2]:
def open_nnet_file(project_root_dir,file_name):
    path=os.path.join(project_root_dir,file_name)
    print("opening file:", file_name)
    file=open(path,"r")
    return file

In [3]:
def get_network_info(record):
    #check to see if the file is structures correctly
    dicti={}
    number_of_inputs=record.readline().strip("\n")
    if (np.isscalar(number_of_inputs)):
        number_of_outputs=record.readline().strip("\n")
        number_of_layers=record.readline().strip("\n")
        number_of_neurons_per_layer=record.readline().strip("\n")
        dicti['number_of_inputs']= int(number_of_inputs)
        dicti['number_of_outputs']=int(number_of_outputs)
        dicti['number_of_layers']=int(number_of_layers)
        dicti['number_of_neurons_in_first_layer']= int(number_of_neurons_per_layer)
    else:
        print("Sherlock file not structured correctly")
    return dicti

In [ ]:
record=open_nnet_file(PROJECT_ROOT_DIR,FILE_NAME)
record
info_dict=get_network_info(record)

In [4]:
def create_nn_matrices(info_dict,record):
    numberOfLayers=info_dict['number_of_layers']
    numberOfInputs=info_dict['number_of_inputs']
    numberOfNeurons=info_dict['number_of_neurons_in_first_layer']
    numberOfOutputs=info_dict['number_of_outputs']
    layerSizes=[numberOfInputs]
    for item in range(0,numberOfLayers):
        layerSizes.append(numberOfNeurons)
    #create layer and weight matrix structure
    info_dict['layer_sizes']=layerSizes
    NN_matrix=[0]*(numberOfLayers+1)
    for layers in range(0,numberOfLayers):
        NN_matrix[layers]=[0]*2
        NN_matrix[layers][0]=np.zeros((layerSizes[layers+1],layerSizes[layers]))
        NN_matrix[layers][1]=np.zeros((layerSizes[layers+1],1))
    #create tthe output layer matrix 
    NN_matrix[numberOfLayers]=[0]*2
    NN_matrix[numberOfLayers][0]=np.zeros((numberOfNeurons,numberOfOutputs))
    NN_matrix[numberOfLayers][1]=np.zeros((numberOfOutputs,1))
    
    #fill the NN_matrix
    for layer in range(0,numberOfLayers+1):
        weight_matrix_shape=NN_matrix[layer][0].shape
        bias_matrix_shape=NN_matrix[layer][1].shape
        for index in range(0,weight_matrix_shape[0]):
            for index2 in range(0,weight_matrix_shape[1]):
                line=record.readline().strip("\n")
                NN_matrix[layer][0][index][index2]=float(line)
        for index3 in range(0,bias_matrix_shape[0]):
            for index4 in range(0,bias_matrix_shape[1]):
                line=record.readline().strip("\n")
                NN_matrix[layer][1][index3][index4]=float(line)
    return NN_matrix
        
    

Divider so I don't get confused

In [5]:
def create_nn_matrices_gen(info_dict,record):
    numberOfLayers=info_dict['number_of_layers']
    numberOfInputs=info_dict['number_of_inputs']
    numberOfNeuronsFirstLayer=info_dict['number_of_neurons_in_first_layer']
    numberOfOutputs=info_dict['number_of_outputs']
    layerSizes=[numberOfInputs,numberOfNeuronsFirstLayer]
    for layer in range(0,numberOfLayers-1):
        line=record.readline().strip("\n")
        layerSizes.append(int(line))
    layerSizes.append(numberOfOutputs)
    info_dict['layer_sizes']=layerSizes
    NN_matrix=[0]*(numberOfLayers+1)
    for layers in range(0,numberOfLayers+1):
        NN_matrix[layers]=[0]*2
        NN_matrix[layers][0]=np.zeros((layerSizes[layers+1],layerSizes[layers]))
        NN_matrix[layers][1]=np.zeros((layerSizes[layers+1],1))
    
    #fill the NN_matrix
    for layer in range(0,numberOfLayers+1):
        weight_matrix_shape=NN_matrix[layer][0].shape
        bias_matrix_shape=NN_matrix[layer][1].shape
        for index in range(0,weight_matrix_shape[0]):
            for index2 in range(0,weight_matrix_shape[1]):
                line=record.readline().strip("\n")
                NN_matrix[layer][0][index][index2]=float(line)
        for index3 in range(0,bias_matrix_shape[0]):
            for index4 in range(0,bias_matrix_shape[1]):
                #print(index3,",",index4)
                line=record.readline().strip("\n")
                NN_matrix[layer][1][index3][index4]=float(line)
    return NN_matrix
            
    

Create function to save it as a matfile

In [23]:
def create_matfile_matrix_dict(NN_matrix):
    numberOfLayers=len(NN_matrix)
    network_weight_matrices=np.zeros((numberOfLayers,), dtype=np.object)
    network_bias_matrices=np.zeros((numberOfLayers,), dtype=np.object)
    for layer in range(0,numberOfLayers):
        network_weight_matrices[layer]=NN_matrix[layer][0]
        network_bias_matrices[layer]=NN_matrix[layer][1]
    return network_weight_matrices, network_bias_matrices
    

In [24]:
def save_mat_file(info_dict,network_weight_matrices,network_bias_matrices,directory_name,file_name):
    NN_matrix_dict={}
    import scipy.io as sio
    import os
    for item in info_dict:
        NN_matrix_dict[item]=info_dict[item]
    NN_matrix_dict['W']=network_weight_matrices
    NN_matrix_dict['b']=network_bias_matrices
    path=os.path.join(directory_name,file_name+".mat")
    sio.savemat(path,NN_matrix_dict)
    return NN_matrix_dict
    
    

In [25]:
record=open_nnet_file(PROJECT_ROOT_DIR,FILE_NAME)
record
info_dict=get_network_info(record)
nn_mat=create_nn_matrices(info_dict,record)
network_weight_matrices1, network_bias_matrices1=create_matfile_matrix_dict(nn_mat)
print((network_weight_matrices1))
print( network_bias_matrices1)

opening file: neural_network_information
[array([[-2.83216e-01, -2.19524e-01],
       [-7.81048e-03, -2.15841e-01],
       [-3.04988e-01,  1.09021e+00],
       [-1.65135e+00,  1.14078e-01],
       [ 7.61641e-01, -6.05356e-01],
       [-3.87565e-01,  9.74517e-02],
       [-2.97733e-01, -4.46726e-01],
       [ 5.55820e-01, -3.11427e-01],
       [-5.99734e-01,  4.49554e-01],
       [-1.11791e+00, -5.87783e-02],
       [ 1.17564e+00, -1.70625e-01],
       [-7.99437e-02,  2.45560e+00],
       [-2.73269e-01, -9.18629e-02],
       [-4.38795e-03, -1.67910e-01],
       [-8.40506e-02,  2.46567e+00],
       [-2.04284e+00,  1.05592e-01],
       [-3.55080e-01, -1.33880e-01],
       [-1.39274e-01,  1.39553e+00],
       [-1.50978e-01, -8.29487e-02],
       [ 2.28684e+00, -1.08054e-02],
       [-4.61647e-01,  1.95655e+00],
       [-5.03975e-03, -3.58779e-01],
       [ 2.52663e+00, -4.46626e-01],
       [-2.02698e-01,  1.21761e-01],
       [ 2.55996e-03, -3.78915e-01],
       [ 2.06058e+00, -3.67221e-0

True

In [26]:
aaaa=save_mat_file(info_dict, network_weight_matrices1, network_bias_matrices1,PROJEC_SAVE_DIR,FILE_NAME)

In [ ]:
nn_mat=create_nn_matrices_gen(info_dict,record)

In [ ]:
nn_mat[7][1]

In [30]:
def decide_which_file_type(record):
    info_dict=get_network_info(record)
    line=record.readline().strip("\n")
    try:
        a=int(line)
        record.close()
        return isinstance(a,int)
    except ValueError:
        record.close()
        return False

In [ ]:
isinstance(6,int)

In [ ]:
record=open_nnet_file(PROJECT_ROOT_DIR,FILE_NAME)
decide_which_file_type(record)

In [35]:
def create_nn_mat_file(nn_source_directory, nn_source_filename, nn_dest_directory,mat_filename):
    record=open_nnet_file(nn_source_directory,nn_source_filename)
    file_type=decide_which_file_type(record)
    if(file_type):
        record=open_nnet_file(nn_source_directory,nn_source_filename)
        info_dict=get_network_info(record)
        nn_mat=create_nn_matrices_gen(info_dict,record)
    else:
        record=open_nnet_file(nn_source_directory,nn_source_filename)
        info_dict=get_network_info(record)
        nn_mat=create_nn_matrices(info_dict,record)
    network_weight_matrices, network_bias_matrices=create_matfile_matrix_dict(nn_mat)
    save_mat_file(info_dict,network_weight_matrices,network_bias_matrices,nn_dest_directory,mat_filename)
        
        
    

In [36]:
create_nn_mat_file(PROJECT_ROOT_DIR,FILE_NAME,PROJEC_SAVE_DIR,FILE_NAME)

opening file: neural_network_information_15
opening file: neural_network_information_15


In [37]:
record.close()

In [38]:
def parse_all_files(nn_source_directory,nn_destination_directory):
    import os
    path=os.path.join(nn_source_directory)
    if(not os.path.exists(path)):
        print ("That directory does not exist")
    else:
        if(not os.path.exists(nn_destination_directory)):
            os.makedirs(nn_destination_directory)
        dirs=os.listdir(path)
        for item in dirs:
            create_nn_mat_file(path,item,nn_destination_directory,item)
            
                
                
        

In [41]:
parse_all_files(PROJECT_ROOT_DIR,PROJEC_SAVE_DIR)

opening file: neural_network_information_10
opening file: neural_network_information_10
opening file: neural_network_information_11
opening file: neural_network_information_11
opening file: neural_network_information_4
opening file: neural_network_information_4
opening file: neural_network_information_15
opening file: neural_network_information_15
opening file: neural_network_information_8
opening file: neural_network_information_8
opening file: neural_network_information_6
opening file: neural_network_information_6
opening file: neural_network_information_14
opening file: neural_network_information_14
opening file: neural_network_information_9
opening file: neural_network_information_9
opening file: neural_network_information_1
opening file: neural_network_information_1
opening file: neural_network_information_7
opening file: neural_network_information_7
opening file: neural_network_information_13
opening file: neural_network_information_13
opening file: neural_network_information_3
o

In [42]:
def load_nn_mat_file(nn_source_dir,file_name):
    import scipy.io as sio
    import os
    if(file_name.endswith(".mat")):
        path=os.path.join(nn_source_dir,file_name)
        mat_contents=sio.loadmat(path,squeeze_me=True)
        del mat_contents['__header__']
        del mat_contents['__version__']
        del mat_contents['__globals__']
        nn_info={}
        network_weights=mat_contents['W']
        network_biases=mat_contents['b']
        del mat_contents['W']
        del mat_contents['b']
        for item in mat_contents.keys():
            nn_info[item]=mat_contents[item]
    else:
        print("File specified is not a .mat file. Please check the directory path")
        return None
    return nn_info, network_weights, network_biases


In [46]:
nn_info,netW,netB=load_nn_mat_file(PROJEC_SAVE_DIR,'neural_network_information.mat')
netB

array([array([ 2.41498e+00,  5.57687e-02, -8.77390e-01,  1.55650e+00,
       -5.00317e-01, -2.65961e-01,  3.21763e-01, -3.13136e-02,
       -5.82646e-01,  1.44951e+00, -4.64122e-01, -3.16251e-01,
        7.25769e-01, -5.24903e-01, -4.32908e-01,  1.87120e-01,
       -1.58243e-01, -8.63952e-02,  2.39618e+00, -4.81356e-01,
       -5.29238e-01,  6.68174e-02, -2.64469e-01, -7.81783e-01,
        4.18599e-01,  8.07436e-03, -5.31674e-01,  2.00818e+00,
        9.18710e-03, -3.55089e-01,  1.94839e+00, -3.97273e-01,
        8.06681e-04,  3.15746e+00, -1.70759e-01, -7.70924e-02,
        2.42419e+00, -3.02577e-03, -3.05363e-01,  2.23606e+00,
       -7.44597e-01, -1.44132e-01,  1.25018e-01, -4.55588e-01,
       -5.85292e-01, -1.09785e-02, -6.57223e-01, -1.45781e-02,
        1.69600e+00, -4.33873e-01, -2.43198e-01, -3.42149e-02,
       -3.77825e-01, -3.34890e-03,  3.20082e+00, -1.26949e+00,
        7.88885e-02,  1.39440e+00, -1.77066e-01, -5.99851e-02,
        2.31845e+00,  2.06645e-02, -3.83438e-01,

In [47]:
#print information in nn_info dictionary
def print_nn_info(nn_info_dict):
    for item in nn_info_dict:
        if(np.isscalar(nn_info_dict[item])):
            print(item+":",nn_info_dict[item])
        else:
            print(item+":",np.array_str(nn_info_dict[item],precision=4))
    print("---------------------------------------")


#load all of the model files in a directory    
def load_all_model_files(nn_source_dir):
    import os
    path=os.path.join(nn_source_dir)
    if(not os.path.exists(path)):
        print("That directory does not exist")
        return None
    else:
        dirs=os.listdir(nn_source_dir)
        dirs2=[]
        for item in dirs:
            if (item.endswith('.mat')):
                dirs2.append(item)
        length=len(dirs2)
        if(length==0):
            print("There are no mat files in that directory")
            return []
        Neural_Networks=[0]*length
        for net in range(0,length):
            Neural_Networks[net]=[0]*3
        for file_name in range(0,length):
            NN_info,NN_weights, NN_biases=load_nn_mat_file(nn_source_dir,dirs2[file_name])
            Neural_Networks[file_name][0]=NN_info
            Neural_Networks[file_name][1]=NN_weights
            Neural_Networks[file_name][2]=NN_info
            
    return Neural_Networks

In [48]:
#print all of the information of the networks in a directory
def print_loaded_networks(list_of_networks):
    length=len(list_of_networks)
    for i in range(0,length):
        print_nn_info(list_of_networks[i][0])

In [54]:
NN_SOURCE=PROJEC_SAVE_DIR
net_list=load_all_model_files(NN_SOURCE)
print_loaded_networks(net_list)

number_of_neurons_in_first_layer: 303
number_of_layers: 24
number_of_inputs: 3
number_of_outputs: 1
layer_sizes: [  3 303 203   4 252   3 303 203   4 252   3 303 203   4 252   3 303 203
   4 252   3 303 203   4 252   1]
---------------------------------------
number_of_neurons_in_first_layer: 100
number_of_layers: 1
number_of_inputs: 2
number_of_outputs: 1
layer_sizes: [  2 100]
---------------------------------------
number_of_neurons_in_first_layer: 303
number_of_layers: 14
number_of_inputs: 3
number_of_outputs: 1
layer_sizes: [  3 303 203   4 252   3 303 203   4 252   3 303 203   4 252   1]
---------------------------------------
number_of_neurons_in_first_layer: 1000
number_of_layers: 1
number_of_inputs: 2
number_of_outputs: 1
layer_sizes: [   2 1000]
---------------------------------------
number_of_neurons_in_first_layer: 200
number_of_layers: 1
number_of_inputs: 2
number_of_outputs: 1
layer_sizes: [  2 200]
---------------------------------------
number_of_neurons_in_first_layer

In [ ]:
np.zeros((2,))